In [1]:
import numpy as np
import cv2
import imutils

In [5]:
check = cv2.imread('newcheck.jpeg', 0)
og = cv2.imread('newcheck.jpeg')

_, check = cv2.threshold(check, 100, 255, cv2.THRESH_BINARY_INV)

canny = cv2.Canny(check, 30, 100)
abc = cv2.cvtColor(check, cv2.COLOR_GRAY2RGB)
temp = og * abc
# canny = cv2.dilate(canny, np.ones((3, 3)),iterations=2)
test = cv2.resize(canny, (500, 500))
cv2.imshow('canny', temp)
cnts = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)
new = check.copy()
cv2.drawContours(check, [c], 0, (0, 255, 0), 50)

contours = [np.array([[100, 180], [200, 280], [200, 180]]), np.array([[280, 70], [12, 20], [80, 150]])]

stencil = np.zeros(new.shape).astype(new.dtype)

# test = cv2.fillPoly(stencil, contours, [255, 255, 255])
test = cv2.resize(check, (500, 500))

result = cv2.bitwise_or(new, stencil)

result = cv2.resize(result, (500, 500))
cv2.imshow('title', test)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
og = cv2.imread('bgwhite.jpeg')
gray = cv2.imread('bgwhite.jpeg', 0)

_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

canny = cv2.Canny(thresh, 30, 100)
canny = cv2.dilate(canny, None,iterations=2)
cnts = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(c)

cropped = og[y:y + h, x:x + w]
cropped_rgb = rgb[y:y + h, x:x + w]

cropped[cropped_rgb < 50] = 0
resized = cv2.resize(cropped, (1000, 1000))
# cv2.imwrite('cropped_segmented.jpg', cropped)

cv2.imshow('title', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()